In [23]:
from datasets import load_dataset
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import average_precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time

In [24]:
# Load the WikiQA dataset
dataset = load_dataset("wiki_qa")
train_data = dataset['train']
valid_data = dataset['validation']
test_data = dataset['test']

# Create DataFrames
train_df = pd.DataFrame({'question': train_data['question'], 'document': train_data['document_title'], 'answer': train_data['answer'], 'label': train_data['label']})
valid_df = pd.DataFrame({'question': valid_data['question'], 'document': valid_data['document_title'], 'answer': valid_data['answer'], 'label': valid_data['label']})
test_df = pd.DataFrame({'question': test_data['question'], 'document': test_data['document_title'], 'answer': test_data['answer'], 'label': test_data['label']})

# Display sample data
print(train_df.head())
print(valid_df.head())
print(test_df.head())

  0%|          | 0/3 [00:00<?, ?it/s]

                        question      document  \
0  how are glacier caves formed?  Glacier cave   
1  how are glacier caves formed?  Glacier cave   
2  how are glacier caves formed?  Glacier cave   
3  how are glacier caves formed?  Glacier cave   
4  how are glacier caves formed?  Glacier cave   

                                              answer  label  
0  A partly submerged glacier cave on Perito More...      0  
1          The ice facade is approximately 60 m high      0  
2          Ice formations in the Titlis glacier cave      0  
3  A glacier cave is a cave formed within the ice...      1  
4  Glacier caves are often called ice caves , but...      0  
                                      question          document  \
0  How are epithelial tissues joined together?  Tissue (biology)   
1  How are epithelial tissues joined together?  Tissue (biology)   
2  How are epithelial tissues joined together?  Tissue (biology)   
3  How are epithelial tissues joined together?  Tissue 

In [25]:
# Text preprocessing
nltk.download('stopwords')
nltk.download('punkt')

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the TF-IDF vectorizer on the combined text of questions and documents
combined_text_train = train_df['question'] + ' ' + train_df['document']
combined_text_valid = valid_df['question'] + ' ' + valid_df['document']
combined_text_test = test_df['question'] + ' ' + test_df['document']

tfidf_train = tfidf_vectorizer.fit_transform(combined_text_train)
tfidf_valid = tfidf_vectorizer.transform(combined_text_valid)
tfidf_test = tfidf_vectorizer.transform(combined_text_test)

# Function to obtain TF-IDF embeddings for a text
def get_tfidf_embedding(text, tfidf_vectorizer=tfidf_vectorizer):
    return tfidf_vectorizer.transform([text]).toarray()[0]

# Apply TF-IDF embedding to the DataFrames
train_df['question_embedding'] = train_df['question'].apply(get_tfidf_embedding)
train_df['document_embedding'] = train_df['document'].apply(get_tfidf_embedding)

valid_df['question_embedding'] = valid_df['question'].apply(get_tfidf_embedding)
valid_df['document_embedding'] = valid_df['document'].apply(get_tfidf_embedding)

test_df['question_embedding'] = test_df['question'].apply(get_tfidf_embedding)
test_df['document_embedding'] = test_df['document'].apply(get_tfidf_embedding)

# Display sample preprocessed data with TF-IDF embeddings
print(train_df.head())
print(valid_df.head())
print(test_df.head())

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
                        question      document  \
0  how are glacier caves formed?  Glacier cave   
1  how are glacier caves formed?  Glacier cave   
2  how are glacier caves formed?  Glacier cave   
3  how are glacier caves formed?  Glacier cave   
4  how are glacier caves formed?  Glacier cave   

                                              answer  label  \
0  A partly submerged glacier cave on Perito More...      0   
1          The ice facade is approximately 60 m high      0   
2          Ice formations in the Titlis glacier cave      0   
3  A glacier cave is a cave formed within the ice...      1   
4  Glacier caves are often called ice caves , but...      0   

                                  question_embedding  \
0  [0.0, 0.0, 0.0, 0.0, 

In [26]:
# Function to perform semantic search on a query using cosine similarity with TF-IDF embeddings
def semantic_search_tfidf(query_embedding, document_embeddings):
    similarities = cosine_similarity([query_embedding], document_embeddings)[0]
    return similarities

# Perform semantic search for 100 queries using TF-IDF embeddings
num_queries = 100
map_scores = []
elapsed_times = []

for i in range(num_queries):
    # Randomly select a query from the test set
    query_row = test_df.sample(1).iloc[0]
    query_embedding = get_tfidf_embedding(query_row['question'])
    
    # Perform semantic search and measure time elapsed
    start_time = time.time()
    predictions = semantic_search_tfidf(query_embedding, tfidf_test)
    elapsed_time = time.time() - start_time
    
    # Calculate MAP for the query
    true_labels = test_df['label'].values
    map_score = calculate_map(predictions, true_labels)  # You need to define the calculate_map function
    # Append results to lists
    map_scores.append(map_score)
    elapsed_times.append(elapsed_time)

# Calculate average MAP and average time elapsed
average_map = np.mean(map_scores)
average_time_elapsed = np.mean(elapsed_times)

# Display results
print(f"Average MAP for {num_queries} queries: {average_map}")
print(f"Average time elapsed for each query: {average_time_elapsed} seconds")

Average MAP for 100 queries: 0.04828070979593003
Average time elapsed for each query: 0.0007987809181213379 seconds
